In [1]:
import cv2
import numpy as np
import os


In [2]:
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


In [3]:
def face_extractor(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        return None

    for (x, y, w, h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face


In [4]:
cap = cv2.VideoCapture(0)
count = 0

while True:
    ret, frame = cap.read()
    
    face = face_extractor(frame)
    
    if face is not None:
        count += 1
        face = cv2.resize(face, (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

        file_name_path = 'dataset/user' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)

        cv2.putText(face, str(count), (50, 50),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

        cv2.imshow('Face Cropper', face)

    else:
        print("Face not found")

    if cv2.waitKey(1) == 13 or count == 100:
        break

cap.release()
cv2.destroyAllWindows()
print("Collecting Samples Complete")


Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found
Face not found


In [5]:
from cv2 import face

data_path = 'dataset/'

onlyfiles = [f for f in os.listdir(data_path)]

Training_Data, Labels = [], []

for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    if images is None:
        continue

    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(0)

Labels = np.asarray(Labels, dtype=np.int32)

model = cv2.face.LBPHFaceRecognizer_create()
model.train(np.asarray(Training_Data), Labels)

print("Model Training Complete")


Model Training Complete


In [6]:
cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()

    face = face_extractor(frame)

    if face is not None:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        face = cv2.resize(face, (200, 200))

        result = model.predict(face)

        if result[1] < 500:
            confidence = int(100 * (1 - result[1] / 300))
            display_string = "Unlocked - " + str(confidence) + "%"
        else:
            display_string = "Locked"

        cv2.putText(frame, display_string, (100, 120),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

    else:
        cv2.putText(frame, "Face Not Found", (100, 120),
                    cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) == 13:
        break

cap.release()
cv2.destroyAllWindows()
